# Without Pipeline
## What we wrote here we need to replicate that in producion code also chances of error too many

In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('train.csv')
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [5]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest  = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)
xtrain

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
459,3,male,NaN,0,0,7.7500,Q
615,2,female,24.0,1,2,65.0000,S
55,1,male,NaN,0,0,35.5000,S
600,2,female,24.0,2,1,27.0000,S
404,3,female,20.0,0,0,8.6625,S
...,...,...,...,...,...,...,...
693,3,male,25.0,0,0,7.2250,C
546,2,female,19.0,1,0,26.0000,S
300,3,female,NaN,0,0,7.7500,Q
417,2,female,18.0,0,2,13.0000,S


## Fixing Missing val

In [7]:
from sklearn.impute import SimpleImputer
si_age = SimpleImputer() # replace by mean
si_embarked = SimpleImputer(strategy='most_frequent') # replace by mode

xtrain_age = si_age.fit_transform(xtrain[['Age']])
xtrain_embarked = si_embarked.fit_transform(xtrain[['Embarked']])

xtest_age = si_age.transform(xtest[['Age']])
xtest_embarked = si_embarked.transform(xtest[['Embarked']])


## OHE

In [8]:
from sklearn.preprocessing import OneHotEncoder
ohe_sex = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_emb = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

xtrain_sex = ohe_sex.fit_transform(xtrain[['Sex']])
xtrain_embarked = ohe_emb.fit_transform(xtrain_embarked)

xtest_sex = ohe_sex.transform(xtest[['Sex']])
xtest_embarked = ohe_emb.transform(xtest_embarked)

xtrain_embarked


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [9]:
xtrain_rem = xtrain.drop(columns=['Age','Sex','Embarked'])
xtest_rem = xtest.drop(columns=['Age','Sex','Embarked'])

In [10]:
xtrain_transformed = np.concatenate((xtrain_rem,xtrain_embarked,xtrain_age,xtrain_sex) , axis=1)
xtest_transformed = np.concatenate((xtest_rem,xtest_embarked,xtest_age,xtest_sex) , axis=1)

xtrain_transformed

array([[ 3.        ,  0.        ,  0.        , ..., 29.51973684,
         0.        ,  1.        ],
       [ 2.        ,  1.        ,  2.        , ..., 24.        ,
         1.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        , ..., 29.51973684,
         0.        ,  1.        ],
       ...,
       [ 3.        ,  0.        ,  0.        , ..., 29.51973684,
         1.        ,  0.        ],
       [ 2.        ,  0.        ,  2.        , ..., 18.        ,
         1.        ,  0.        ],
       [ 3.        ,  0.        ,  0.        , ..., 24.        ,
         1.        ,  0.        ]])

In [11]:
xtrain_transformed.shape

(712, 10)

In [12]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

clf.fit(xtrain_transformed,ytrain)

DecisionTreeClassifier()

In [13]:
y_pred = clf.predict(xtest_transformed)
y_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0], dtype=int64)

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,y_pred)*100

79.3296089385475

# With Pipeline

## steps:
1. impute missing vals
2. ohe
3. Scaling
4. Feature Selection : taking only relevant cols
5. Decision Tree algo

# The above steps are our pipeline


In [15]:
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier

from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
df1 = pd.read_csv('train.csv')
df1.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df1.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [16]:
df1

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [17]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest  = train_test_split(df1.iloc[:,1:],df1.iloc[:,0],test_size=0.2)
Xtrain

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
786,3,female,18.0,0,0,7.4958,S
851,3,male,74.0,0,0,7.7750,S
749,3,male,31.0,0,0,7.7500,Q
757,2,male,18.0,0,0,11.5000,S
860,3,male,41.0,2,0,14.1083,S
...,...,...,...,...,...,...,...
495,3,male,NaN,0,0,14.4583,C
834,3,male,18.0,0,0,8.3000,S
818,3,male,43.0,0,0,6.4500,S
159,3,male,NaN,8,2,69.5500,S


### impute 
## Note: use col index in col transformer it is better

In [18]:
trf1 = ColumnTransformer(transformers=[('si_age',SimpleImputer(),[2]),
                         ('si_embarked',SimpleImputer(strategy='most_frequent'),[6])],remainder='passthrough')

### OHE

In [19]:
trf2 = ColumnTransformer(transformers=[('ohe_sex_embarked',OneHotEncoder(sparse=False,dtype=np.int32,handle_unknown='ignore'),[1,6])],remainder='passthrough')

### Scaling 
## below is how to apply on all cols

## before was 0 - 7 cols now after above trans = 0 - (7 - 2 + 2(male/female) + 3(embarked) ) = 0 - 10

In [20]:
trf3 = ColumnTransformer([('scale',MinMaxScaler(),slice(0,10))])

### Feature Selection :: Pick 8 best cols

In [21]:
trf4 = SelectKBest(score_func=chi2,k = 8)

### DT

In [22]:
trf5 = DecisionTreeClassifier()


## Pipeline

In [23]:
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

pipe.fit(Xtrain,Ytrain)

c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('si_age', SimpleImputer(),
                                                  [2]),
                                                 ('si_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(dtype=<class 'numpy.int32'>,
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000002720E5B2A20>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [24]:
Ypred = pipe.predict(Xtest)
Ypred

array([1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0], dtype=int64)

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(Ytest,Ypred)*100

63.128491620111724

# Explore Pipes

In [26]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('si_age', SimpleImputer(), [2]),
                                 ('si_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(dtype=<class 'numpy.int32'>,
                                                handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x000002720E5B2A20>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [27]:
pipe.named_steps['columntransformer-1'].transformers_

[('si_age', SimpleImputer(), [2]),
 ('si_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [28]:
pipe.named_steps['columntransformer-1'].transformers_[0]

('si_age', SimpleImputer(), [2])

In [29]:
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_

array([29.8627193])

# Cross Validation Pipeline

In [30]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,Xtrain,Ytrain,cv=5,scoring='accuracy').mean()*100

c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\

63.76538954003743

# Exporting Pipeline

In [31]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))